In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dspy

lm = dspy.LM('ollama_chat/gpt-oss:20b')
dspy.configure(lm=lm)

In [3]:
from pydantic import BaseModel, Field

class Function(BaseModel):
    code: str = Field(
        ..., 
        description=(
            ("Code for the function. Include docstrings as well as type hints for input arguments and expected outputs. "),
            ("Include any import statements (if any) within the function's code itself.")
        )
    )
    
class Step(BaseModel):
    step_order: int = Field(..., description="Order of step. For example: 1, 2, 3. Step number 1 will always be executed first")
    step_description: str = Field(..., description="What this step aims to fulfil")
    function: Function = Field(..., description="The python function")

class Plan(dspy.Signature):
    question: str = dspy.InputField()
    steps: list[Step] = dspy.OutputField(desc="The steps and their associated Python functions to adress the user's question.")
    
class WriteTool(dspy.Signature):
    code: str = dspy.InputField()
    repurposed_code: str = dspy.OutputField(
        desc="Consolidate these codes into 1 function that invokes it. This function will be used as a tool to an LLM so include arguments, type hints and dosctrings. Double check to ensure that you've imported everything you need for the tool to work."
    )

class ToolWriter(dspy.Module):
    def __init__(self):
        super().__init__()
        self.planner = dspy.ChainOfThought(Plan)
        self.format_to_tool = dspy.ChainOfThought(WriteTool)
    
    def consolidate_step_codes(self, plan: Plan):
        return_ = ""
        for step in plan.steps:
            return_ += step.function.code
        return return_
    
    def forward(self, question: str):
        plan = self.planner(question=question)
        code = self.consolidate_step_codes(plan=plan)
        tool = self.format_to_tool(code=code)
        return tool

In [4]:
import random

to_sort = [random.randint(1, 100) for _ in range(1000)]

In [5]:
question = f"Do a heap sort of the following list: {to_sort}"
tool_writer = ToolWriter()
tool = tool_writer(question=question)

In [6]:
from IPython.display import display, Markdown

display(Markdown(tool.repurposed_code))

def heap_sort_tool(arr: list[int]) -> list[int]:
    """
    Sort a list of integers using the heap sort algorithm.

    This function performs an in‑place heap sort on the provided list and
    returns the sorted list in ascending order.

    Args:
        arr: A list of integers to be sorted.

    Returns:
        The same list instance, sorted in ascending order.
    """
    def sift_down(a: list[int], start: int, end: int) -> None:
        root = start
        while True:
            child = 2 * root + 1
            if child >= end:
                break
            if child + 1 < end and a[child] < a[child + 1]:
                child += 1
            if a[root] < a[child]:
                a[root], a[child] = a[child], a[root]
                root = child
            else:
                break

    n = len(arr)
    for i in range(n // 2 - 1, -1, -1):
        sift_down(arr, i, n)
    for end in range(n - 1, 0, -1):
        arr[0], arr[end] = arr[end], arr[0]
        sift_down(arr, 0, end)
    return arr

In [7]:
tower_of_hanoi = """
Bob and Alice like to play the game tower of Hanoi. One day Alice challenges Bob to build the tallest tower from a set of disks of different height and radius. 

The tower of Hanoi can be built by stacking disks on top of each other. In order to put disk A on top of disk B, the radius and height of A must be strictly smaller 
than those of B. Help Bob to win the challenge.

Input:
First line of input contains number of test cases T.
First line of each test case contains value of N, the number of disks. The next N lines contains two positive integer number Ri and Hi corresponding to the radius and height of ith Disk respectively.

Output:
For each test case print the maximum height of the tower possible.

Constraints:
1<=T<=10
1<=N<=200
1<=Ri, Hi<=10^9
"""

tower_of_hanoi_tool = tool_writer(question=tower_of_hanoi)

In [8]:
display(Markdown(tower_of_hanoi_tool.repurposed_code))

```python
from __future__ import annotations
from typing import List, Tuple


def compute_tower_heights(input_data: str) -> List[int]:
    """
    Compute the tallest possible tower height for each test case described in the
    input string.

    The input format follows the problem statement:
        - The first line contains an integer T, the number of test cases.
        - For each test case:
            * A line with an integer N, the number of disks.
            * N subsequent lines, each containing two integers R and H
              representing the radius and height of a disk.

    The function parses the input, applies a weighted longest‑increasing‑subsequence
    algorithm on the heights after sorting by radius (ascending) and height
    (descending for equal radii), and returns a list of the maximum tower heights
    for each test case.

    Parameters
    ----------
    input_data : str
        Raw input data as a single string, typically read from a file or
        provided directly.

    Returns
    -------
    List[int]
        A list where each element is the maximum tower height for the
        corresponding test case. If the input is empty or contains no
        test cases, an empty list is returned.

    Examples
    --------
    >>> data = \"\"\"2\n3\n2 3\n1 4\n3 2\n2\n1 5\n1 5\"\"\"\n>>> compute_tower_heights(data)\n[7, 5]\n    """
    # ---------- Parsing ----------
    tokens = input_data.strip().split()
    if not tokens:
        return []

    it = iter(tokens)
    try:
        t = int(next(it))
    except StopIteration:
        return []

    test_cases: List[Tuple[int, List[Tuple[int, int]]]] = []

    for _ in range(t):
        try:
            n = int(next(it))
        except StopIteration:
            break

        disks: List[Tuple[int, int]] = []
        for _ in range(n):
            try:
                r = int(next(it))
                h = int(next(it))
            except StopIteration:
                break
            disks.append((r, h))

        test_cases.append((n, disks))

    # ---------- Solving ----------
    results: List[int] = []

    for n, disks in test_cases:
        # Sort by radius ascending, height descending to enforce strictness on equal radii
        disks.sort(key=lambda x: (x[0], -x[1]))
        heights = [h for _, h in disks]

        dp: List[int] = [0] * n
        for i in range(n):
            dp[i] = heights[i]  # at least the disk itself
            for j in range(i):
                if heights[j] < heights[i]:
                    dp[i] = max(dp[i], dp[j] + heights[i])

        results.append(max(dp) if dp else 0)

    return results
```